# Movie Reviews and Bag-of-Words Modelling

🎯 The goal of this challenge is to play with the ***Bag-of-words*** modelling of texts.

✍️ In the following dataset, we have $2000$ reviews classified either as _"positive"_ or _"negative"_.

In [42]:
import string
string.punctuation
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
import pandas as pd

data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/10-Natural-Language-Processing/movie_reviews.csv")
data.head()

,target,reviews
0,neg,"plot : two teen couples go to a church party ,..."
1,neg,the happy bastard's quick movie review \ndamn ...
2,neg,it is movies like these that make a jaded movi...
3,neg,""" quest for camelot "" is warner bros . ' firs..."
4,neg,synopsis : a mentally unstable man undergoing ...


In [17]:
data.shape

(2000, 2)

## 1. Preprocessing

❓ **Question (Cleaning Text)** ❓

- Write a function `preprocessing` that will clean a sentence and apply it to all our reviews. It should:
    - remove whitespace
    - lowercase characters
    - remove numbers
    - remove punctuation
    - tokenize
    - lemmatize
- You can store the cleaned reviews into a column called `cleaned_reviews`.
- Do not remove stopwords in this challenge, we will explain why in the section `3. N-gram modelling`

In [18]:
import string
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

In [36]:
def preprocessing(text):
    # Lowercase characters
    text = text.lower()
    # Remove numbers
    text = ''.join(char for char in text if not char.isdigit())
    # Remove punctuation
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '') 
    # Remove whitespace
    text = text.strip()
    
    # Tokenize 
    token = word_tokenize(text)
    # Lemmatize
    text_cleaned = [WordNetLemmatizer().lemmatize(word) for word in token]
    return text_cleaned

data['cleaned_reviews'] = data['reviews'].apply(preprocessing)
data['cleaned_reviews'] = data['cleaned_reviews'].apply(lambda x: ', '.join([str(i) for i in x]))

❓ **Question (LabelEncoding)**❓

LabelEncode your target and store it into a column called `"target_encoded"`

In [20]:
from sklearn.preprocessing import LabelEncoder

In [37]:
target = data['target']
target.value_counts()

neg    1000
pos    1000
Name: target, dtype: int64

In [39]:
encoder = LabelEncoder()
encoder.fit_transform(target)
data['target_encoded'] = encoder.fit_transform(target)

In [38]:
# Quick check
data.head()

,target,reviews,cleaned_reviews,target_encoded
0,neg,"plot : two teen couples go to a church party ,...","plot, two, teen, couple, go, to, a, church, pa...",0
1,neg,the happy bastard's quick movie review \ndamn ...,"the, happy, bastard, quick, movie, review, dam...",0
2,neg,it is movies like these that make a jaded movi...,"it, is, movie, like, these, that, make, a, jad...",0
3,neg,""" quest for camelot "" is warner bros . ' firs...","quest, for, camelot, is, warner, bros, first, ...",0
4,neg,synopsis : a mentally unstable man undergoing ...,"synopsis, a, mentally, unstable, man, undergoi...",0


## 2. Bag-of-Words Modelling

❓ **Question (NaiveBayes with unigrams)** ❓

Using `cross_validate`, score a Multinomial Naive Bayes model trained on a Bag-of-Words representation of the texts.

In [41]:
count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(data['cleaned_reviews'])
X_array = X.toarray()

In [44]:
# Feature/Target
X = X_array
y = data["target_encoded"]

# Naive Bayes
pipeline_naive_bayes = make_pipeline(
    MultinomialNB()
)

# Cross-validation
cv_results = cross_validate(pipeline_naive_bayes, X, y, cv = 5)
accuracy = cv_results['test_score'].mean()
accuracy

0.8160000000000001

## 3. N-gram Modelling

👀 Remember that we asked you not to remove stopwords. Why? 

👉 We will train the Naive Bayes model with bigrams. Hence, in sentence like "I do not like coriander", it is important to scan the bigram "do not" to detect negativity in this sentence for example.

❓ **Question (NaiveBayes with bigrams)** ❓

Using `cross_validate`, score a Multinomial Naive Bayes model trained on a 2-gram Bag-of-Words representation of the texts.

In [46]:
vectorizer = CountVectorizer(ngram_range = (2,2))
naivebayes = MultinomialNB()

X_bow = vectorizer.fit_transform(data.cleaned_reviews)

cv_nb = cross_validate(
    naivebayes,
    X_bow,
    data.target_encoded,
    scoring = "accuracy"
)

round(cv_nb['test_score'].mean(),2)

0.84

🏁 Congratulations! Now, you know how to train a Naive Bayes model on vectorized texts.

💾 Don't forget to `git add/commit/push` your notebook...

🚀 ... and move on to the next challenge!